<a href="https://colab.research.google.com/github/FredLongo/DataMiningProject/blob/main/code/GetTwitterSentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This first part is about processing the Twitter data to a format we can work with.   


In [80]:
import json


In [81]:

#with open("twiterscraper.json", "r") as f:
#infile = "/content/dataset_easy-twitter-search-scraper_2023-11-21_01-10-14-456.json"
infile = "/content/dataset_easy-twitter-search-scraper_2023-11-21_18-49-26-401.json"
# Data pulled from https://console.apify.com/actors/2s3kSMq7tpuC3bI6M/runs/cMp5HramffKLAmVSt#output

with open(infile, "r") as f:
  data = json.load(f)



In [82]:
'''
Set up Username Counts
'''

username =["@USNewsMoney", "@TMFStockAdvisor", "@ftfinancenews", "@Stocktwits", "@MadMoneyOnCNBC", "@SquawkCNBC"]

cols = 2
rows = len(username)
username_counts = [[0 for _ in range(cols)] for _ in range(rows)]

# Fill the first column with values from the list
for i in range(rows):
    username_counts[i][0] = username[i]

# Printing the 2D array
#for row in username_counts:    print(row)


In [83]:
'''
Set up selectQ Counts
'''
searchQuery = ["stock","Apple","Microsoft","Alphabet","Amazon","Nvidia","Tesla","GOOG","AMZN","AAPL","META","MSFT","NVDA","TSLA"]

cols = 2
rows = len(searchQuery)
searchQuery_counts = [[0 for _ in range(cols)] for _ in range(rows)]

# Fill the first column with values from the list
for i in range(rows):
    searchQuery_counts[i][0] = searchQuery[i]




In [ ]:
'''

x_count = 0 #120
user_count = 0   #2900

for x in data:
  x_count+=1

  #get username_counts
  username_counts[username.index(x['user']['username'].rstrip())][1]+=1
  Qu = username_counts[username.index(x['user']['username'].rstrip())][1]

  #get searchQuery_counts
  searchQuery_counts[searchQuery.index(x['searchQuery'].rstrip())][1]+=1
  Qv = searchQuery_counts[searchQuery.index(x['searchQuery'].rstrip())][1]

  print(x_count, x['searchQuery'] ,'=', Qv,x['user']['username'],'=',Qu,'  text:',x['text'])

'''

'''
  Reviewing Data in file
  print('>>>',x_count,'   >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>')
  print('id:',x['id'])
  print('url:',x['url'])
  print('verified:',x['verified'])
  print('timestamp:',x['timestamp'])
  print('text:',x['text'])
  print('links:',x['links'])
  print('isQuote:',x['isQuote'])
  print('isRetweet:',x['isRetweet'])
  print('likes:',x['likes'])
  print('replies:',x['replies'])
  print('retweets:',x['retweets'])
  print('quotes:',x['quotes'])
  print('searchQuery:',x['searchQuery'])
  print('user:',x['user']['username'])
  print('\n\n')
'''

In [ ]:
'''
for row in searchQuery_counts:
    print(row)

print("\n")

for row in username_counts:
    print(row)
'''

['stock', 0]
['Apple', 23]
['Microsoft', 14]
['Alphabet', 5]
['Amazon', 0]
['Nvidia', 16]
['Tesla', 21]
['GOOG', 3]
['AMZN', 39]
['AAPL', 29]
['META', 39]
['MSFT', 22]
['NVDA', 39]
['TSLA', 42]


['@USNewsMoney', 4]
['@TMFStockAdvisor', 0]
['@ftfinancenews', 6]
['@Stocktwits', 174]
['@MadMoneyOnCNBC', 8]
['@SquawkCNBC', 100]


In [73]:
'''
out_data = []
for x in data:
  out_data.append(
        {"timestamp":x['timestamp']
        ,"username":x['user']['username']
        ,"searchQuery":x['searchQuery']
        ,"sentement":""
        ,"score":""
        ,"text":x['text']})
'''





At this point I want to add the code to do the sentiment alalisis.



In [84]:

from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax


# load model and tokenizer
roberta = "cardiffnlp/twitter-roberta-base-sentiment"

model = AutoModelForSequenceClassification.from_pretrained(roberta)
tokenizer = AutoTokenizer.from_pretrained(roberta)

labels = ['Negative', 'Neutral', 'Positive']



In [85]:
def preprocesses_tweet(tweet):
  tweet_words = []

  # replace
  for word in tweet.split(' '):
    # username
    if word.startswith('@') and len(word) > 1 :
      word = '@user'

    # http link
    elif word.startswith('http'):
      word = "http"

    elif word == '':
      continue

    tweet_words.append(word)

  tweet_proc = " ".join(tweet_words)
  return tweet_proc


In [86]:
def get_tweet_sentiment(tweet_proc):
  # sentiment analysis
  encoded_tweet = tokenizer(tweet_proc, return_tensors='pt')
  # output = model(encoded_tweet['input_ids'], encoded_tweet['attention_mask'])
  output = model(**encoded_tweet)

  scores = output[0][0].detach().numpy()
  scores = softmax(scores)

  maxposition = 0
  maxscore = 0

  for i in range(len(scores)):
    if maxscore <= scores[i]:
      maxposition = i
      maxscore = scores[i]

  return labels[maxposition]  , scores[maxposition]

#  for i in range(len(scores)):
#    l = labels[i]
#    s = scores[i]
#    print(l,s)



In [97]:
out_data = []

# Need to create a tweet loop

for x in data:
  tweet_proc = preprocesses_tweet(x['text'])
  sentiment_label, sentiment_score = get_tweet_sentiment(tweet_proc)
  out_data.append(
        {"timestamp":x['timestamp']
        ,"username":x['user']['username']
        ,"searchQuery":x['searchQuery']
        ,"sentement":sentiment_label
        ,"score":str(sentiment_score)
        ,"text":tweet_proc})


  #print(x['timestamp'], x['user']['username'],x['searchQuery'],sentiment_label, sentiment_score, tweet_proc)


    #break  # Exit the loop... for testing




In [98]:
outfilename = infile[:-5] + "_out.json"

with open(outfilename, "w") as file:
    json.dump(out_data, file)